In [1]:
import math
import re
import random


class Example(object):
    def __init__(self, attributes, classification, weight = 1.0):
        self.attributes = attributes
        self.classification = classification
        self.weight = weight


class Attribute(object):
    def __init__(self, name, domain):
        self.name = name
        self.domain = domain


class DecisionTree(object):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name
        self.branches = dict()

    def add_a_branch(self, vk, subtree, weight):
        self.branches[vk] = (subtree, weight)


def PLURALITY_VALUE(examples):
    count = dict()
    for example in examples:
        classification = example.classification
        if count.get(classification) is None:
            count[classification] = example.weight
        else:
            count[classification] += example.weight
    return max(count, key=lambda k: count[k]), len(count) == 1


def B(q):
    if q <= 0 or q >= 1:
        return 0.0
    return -(q * math.log(q, 2) + (1 - q) * math.log(1 - q, 2))


def INFORMATION_COUNT(a, examples):
    positive = examples[0].classification
    p = 0.0
    n = 0.0
    pk = dict()
    nk = dict()
    real_p = 0.0
    real_n = 0.0
    for value in a.domain:
        pk[value] = 0.0
        nk[value] = 0.0
    for example in examples:
        value = example.attributes[a.name]
        if positive == example.classification:
            real_p += example.weight
            if value != '?':
                p += example.weight
                pk[value] += example.weight
        else:
            real_n += example.weight
            if value != '?':
                n += example.weight
                nk[value] += example.weight
    if p != 0:
        t0 = real_p / p
    if n != 0:
        t1 = real_n / n
    for value in a.domain:
        if p != 0:
            pk[value] *= t0
        else:
            pk[value] = real_p / len(a.domain)
        if n != 0:
            nk[value] = nk[value] * t1
        else:
            nk[value] = real_n / len(a.domain)
        nk[value] += pk[value]
    return real_p, real_p + real_n, pk, nk


def GAIN(a, examples):
    p, p_add_n, pk, pk_add_nk = INFORMATION_COUNT(a, examples)
    remainder = 0
    for value in a.domain:
        if pk_add_nk[value] != 0:
            remainder += pk_add_nk[value] / p_add_n * B(pk[value] / pk_add_nk[value])
    return B(p / p_add_n) - remainder


def GAIN_RATIO(a, examples):
    p, p_add_n, pk, pk_add_nk = INFORMATION_COUNT(a, examples)
    remainder = 0
    split_information = 0
    for value in a.domain:
        if pk_add_nk[value] != 0:
            remainder += pk_add_nk[value] / p_add_n * B(pk[value] / pk_add_nk[value])
            split_information += B(pk_add_nk[value] / p_add_n)
    b = B(p / p_add_n)
    if b == 0:
        return 0
    if split_information == 0:
        return float("inf") * (b - remainder)
    return (b - remainder) / split_information


def CLASSIFICATION_COUNT(classification, examples):
    count = 0
    for example in examples:
        if classification == example.classification:
            count += example.weight
    return count


def WEIGHTING(exs_dict, A, examples):
    total_weight = 0
    weights = dict()
    for vk in A.domain:
        exs_dict[vk] = list()
        weights[vk] = 0
    unknown_list = list()
    for e in examples:
        A_value = e.attributes[A.name]
        if A_value == '?':
            unknown_list.append(e)
        else:
            exs_dict[A_value].append(e)
            total_weight += e.weight
            weights[A_value] += e.weight
    real_weights = {k: v for (k, v) in weights.items()}
    for example in unknown_list:
        for vk in exs_dict:
            if total_weight != 0:
                weight = example.weight * weights[vk] / total_weight
            else:
                weight = example.weight / len(examples)
            exs_dict[vk].append(Example(example.attributes, example.classification, weight))
    return real_weights


def DECISION_TREE_LEARNING(training_examples, attributes, IMPORTANCE, preprune, validating_examples = None, weight = None, split_point = 0.7):
    if len(training_examples) == 0 or weight == 0:
        return None
    plurality_value, same_classification = PLURALITY_VALUE(training_examples)
    if same_classification or len(attributes) == 0:
        return plurality_value
    if preprune is True and validating_examples is None:
        random.shuffle(training_examples)
        split_point = int(len(training_examples) * split_point)
        validating_examples = training_examples[split_point:]
        training_examples = training_examples[:split_point]
    A = max(attributes, key=lambda a: IMPORTANCE(a, training_examples))
    tree = DecisionTree(A.name)
    exs_dict = dict()
    exs_vali_dict = dict()
    weights = WEIGHTING(exs_dict, A, training_examples)
    if preprune is True:
        last_right = CLASSIFICATION_COUNT(plurality_value, validating_examples)
        WEIGHTING(exs_vali_dict, A, validating_examples)
        right = 0
        for exs in exs_vali_dict.values():
            if len(exs) != 0:
                right += CLASSIFICATION_COUNT(PLURALITY_VALUE(exs)[0], exs)
        if last_right >= right:
            return plurality_value
    attributes.remove(A)
    for vk in A.domain:
        subtree = DECISION_TREE_LEARNING(exs_dict[vk], attributes, IMPORTANCE, preprune, exs_vali_dict.get(vk), weights[vk])
        if subtree is None:
            subtree = plurality_value
        tree.add_a_branch(vk, subtree, weights[vk])
    attributes.append(A)
    return tree


def LOAD_ATTRIBUTES(path):
    attributes = list()
    continuous_indexes = list()
    with open(path) as f:
        for i in range(0, 96):
            f.readline()
        for i in range(96, 110):
            l = re.findall(r'[^:,\.\s]+', f.readline())
            if l[1:] == ['continuous']:
                continuous_indexes.append(len(attributes))
                attributes.append(Attribute(l[0], list()))
            else:
                attributes.append(Attribute(l[0], l[1:]))
    return attributes, continuous_indexes


def GAIN_FOR_BI_PARTITION(a_name, mid, examples):
    positive = examples[0].classification
    p = 0.0
    p_add_n = 0.0
    pk = {False: 0.0, True: 0.0}
    pk_add_nk = {False: 0.0, True: 0.0}
    for example in examples:
        value = example.attributes[a_name]
        p_add_n += example.weight
        pk_add_nk[float(value) <= mid] += example.weight
        if positive == example.classification:
            p += example.weight
            pk[float(value) <= mid] += example.weight
    remainder = 0
    for value in (False, True):
        if pk_add_nk[value] != 0:
            remainder += pk_add_nk[value] / p_add_n * B(pk[value] / pk_add_nk[value])
    return B(p / p_add_n) - remainder


def LOAD_TRAINING_EXAMPLES(path, weighting):
    training_examples = list()
    with open(path) as f:
        line = f.readline()
        while line != '\n':
            l = re.findall(r'[^,\s]+', line)
            if weighting or '?' not in l:
                example = Example({attributes[i].name: l[i] for i in range(len(attributes))}, l[-1])
                training_examples.append(example)
            line = f.readline()
    return training_examples


def CONTINUOUS_VALUE_PROCESSING(continuous_indexes, training_examples, attributes, is_bi_partition):
    continuous_values = dict()
    continuous_mid = dict()
    for example in training_examples:
        for index in continuous_indexes:
            if continuous_values.get(index) is None:
                continuous_values[index] = set()
            else:
                continuous_values[index].add(example.attributes[attributes[index].name])
    for index in continuous_indexes:
        name = attributes[index].name
        continuous_values[index] = list(continuous_values[index])
        continuous_values[index].sort()
        continuous_mid[index] = list()
        for i in range(len(continuous_values[index]) - 1):
            continuous_mid[index].append((float(continuous_values[index][i]) + float(continuous_values[index][i + 1])) / 2)
        if is_bi_partition is True:
            continuous_mid[index] = [max(continuous_mid[index][::int(len(continuous_values[index]) / 1000) + 1], key = lambda mid: GAIN_FOR_BI_PARTITION(name, mid, training_examples))]
        for i in range(len(continuous_mid[index]) + 1):
            attributes[index].domain.append(str(i))
    return continuous_mid


def TRAINING_EXAMPLES_UPDATE(training_examples, continuous_indexes, continuous_mid, attributes):
    for example in training_examples:
        for index in continuous_indexes:
            i = 0
            while i < len(continuous_mid[index]) and float(example.attributes[attributes[index].name]) > continuous_mid[index][i]:
                i += 1
            example.attributes[attributes[index].name] = str(i)


def DECISION_TREE_PREDICTING(example_attributes, tree):
    predict = list()
    stack = [(tree, 1)]
    while len(stack) > 0:
        classification, weight = stack.pop()
        if type(classification) == DecisionTree:
            for attribute_name in example_attributes:
                if attribute_name == classification.attribute_name:
                    value = example_attributes[attribute_name]
                    if value == '?':
                        for new_classification, new_weight in classification.branches.values():
                            stack.append((new_classification, weight * new_weight))
                    else:
                        stack.append((classification.branches[value][0], weight))
                    break
        else:
            predict.append((classification, weight))
    return predict


def DECISION_TREE_TESTING(path, decision_tree, continuous_indexes, continuous_mid, attributes):
    TP = 0.0
    FP = 0.0
    TN = 0.0
    FN = 0.0
    positive = None
    with open(path) as f:
        f.readline()
        line = f.readline()
        while line != '\n':
            l = re.findall(r'[^,.\s]+', line)
            example_attributes = {attributes[i].name: l[:-1][i] for i in range(len(attributes))}
            for index in continuous_indexes:
                i = 0
                while i < len(continuous_mid[index]) and float(l[index]) > continuous_mid[index][i]:
                    i += 1
                example_attributes[attributes[index].name] = str(i)
            if positive is None:
                positive = l[-1]
            for classification, weight in DECISION_TREE_PREDICTING(example_attributes, decision_tree):
                if l[-1] == positive:
                    if classification == positive:
                        TP += weight
                    else:
                        FP += weight
                else:
                    if classification != positive:
                        TN += weight
                    else:
                        FN += weight
            line = f.readline()
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * precision * recall / (precision + recall)
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1-score: ', f1_score)

In [2]:
print('N-partition without weighting:')
attributes, continuous_indexes = LOAD_ATTRIBUTES("adult.names")
training_examples = LOAD_TRAINING_EXAMPLES("adult.data", False)
%time continuous_mid = CONTINUOUS_VALUE_PROCESSING(continuous_indexes, training_examples, attributes, False)
TRAINING_EXAMPLES_UPDATE(training_examples, continuous_indexes, continuous_mid, attributes)

N-partition without weighting:
Wall time: 81.8 ms


In [3]:
print('ID3:')
%time ID3_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, False)
DECISION_TREE_TESTING('adult.test', ID3_tree, continuous_indexes, continuous_mid, attributes)

ID3:
Wall time: 1min 59s
Accuracy:  0.5839375996797335
Precision:  0.6231022592764139
Recall:  0.7799549065922268
F1-score:  0.692761031059573


In [4]:
print('C4.5:')
%time C45_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, False)
DECISION_TREE_TESTING('adult.test', C45_tree, continuous_indexes, continuous_mid, attributes)

C4.5:
Wall time: 2min 17s
Accuracy:  0.15849483366607361
Precision:  0.5465942887427928
Recall:  0.17832970726612066
F1-score:  0.26892198366030406


In [5]:
print('ID3 with prepruning:')
%time ID3_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, True)
DECISION_TREE_TESTING('adult.test', ID3_prepruning_tree, continuous_indexes, continuous_mid, attributes)

ID3 with prepruning:
Wall time: 375 ms
Accuracy:  0.7637737239727289
Precision:  1.0
Recall:  0.7637737239727289
F1-score:  0.8660676974508984


In [6]:
print('C4.5 with prepruning:')
%time C45_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, True)
DECISION_TREE_TESTING('adult.test', C45_prepruning_tree, continuous_indexes, continuous_mid, attributes)

C4.5 with prepruning:
Wall time: 16.7 s
Accuracy:  0.7428642183364261
Precision:  0.9590141247477724
Recall:  0.7521971750106795
F1-score:  0.8431077045043348


In [7]:
print('2-partition without weighting:')
attributes, continuous_indexes = LOAD_ATTRIBUTES("adult.names")
training_examples = LOAD_TRAINING_EXAMPLES("adult.data", False)
print('2-partition:')
%time continuous_mid = CONTINUOUS_VALUE_PROCESSING(continuous_indexes, training_examples, attributes, True)
TRAINING_EXAMPLES_UPDATE(training_examples, continuous_indexes, continuous_mid, attributes)

2-partition without weighting:
2-partition:
Wall time: 41.7 s


In [8]:
print('ID3:')
%time ID3_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, False)
DECISION_TREE_TESTING('adult.test', ID3_tree, continuous_indexes, continuous_mid, attributes)

ID3:
Wall time: 2.9 s
Accuracy:  0.5136015182591623
Precision:  0.584676920793097
Recall:  0.7731809037862272
F1-score:  0.6658444232654586


In [9]:
print('C4.5:')
%time C45_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, False)
DECISION_TREE_TESTING('adult.test', C45_tree, continuous_indexes, continuous_mid, attributes)

C4.5:
Wall time: 2.91 s
Accuracy:  0.5021240523195487
Precision:  0.7365208005180185
Recall:  0.3780254915578007
F1-score:  0.49961789768249215


In [10]:
print('ID3 with prepruning:')
%time ID3_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, True)
DECISION_TREE_TESTING('adult.test', ID3_prepruning_tree, continuous_indexes, continuous_mid, attributes)

ID3 with prepruning:
Wall time: 349 ms
Accuracy:  0.7637737239727289
Precision:  1.0
Recall:  0.7637737239727289
F1-score:  0.8660676974508984


In [11]:
print('C4.5 with prepruning:')
%time C45_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, True)
DECISION_TREE_TESTING('adult.test', C45_prepruning_tree, continuous_indexes, continuous_mid, attributes)

C4.5 with prepruning:
Wall time: 844 ms
Accuracy:  0.8177968702055846
Precision:  0.9934934532894208
Recall:  0.81069743051914
F1-score:  0.8928352282981411


In [12]:
print('N-partition with weighting:')
attributes, continuous_indexes = LOAD_ATTRIBUTES("adult.names")
training_examples = LOAD_TRAINING_EXAMPLES("adult.data", True)
%time continuous_mid = CONTINUOUS_VALUE_PROCESSING(continuous_indexes, training_examples, attributes, False)
TRAINING_EXAMPLES_UPDATE(training_examples, continuous_indexes, continuous_mid, attributes)

N-partition with weighting:
Wall time: 89.8 ms


In [13]:
print('ID3:')
%time ID3_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, False)
DECISION_TREE_TESTING('adult.test', ID3_tree, continuous_indexes, continuous_mid, attributes)

ID3:
Wall time: 3min 26s
Accuracy:  0.5952553047444487
Precision:  0.677929340817544
Recall:  0.7327200875235532
F1-score:  0.7042606560621736


In [14]:
print('C4.5:')
%time C45_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, False)
DECISION_TREE_TESTING('adult.test', C45_tree, continuous_indexes, continuous_mid, attributes)

C4.5:
Wall time: 5min 8s
Accuracy:  0.33067559095887333
Precision:  0.6179401043819271
Recall:  0.39742237104790795
F1-score:  0.4837350746985091


In [15]:
print('ID3 with prepruning:')
%time ID3_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, True)
DECISION_TREE_TESTING('adult.test', ID3_prepruning_tree, continuous_indexes, continuous_mid, attributes)

ID3 with prepruning:
Wall time: 389 ms
Accuracy:  0.7637737239727289
Precision:  1.0
Recall:  0.7637737239727289
F1-score:  0.8660676974508984


In [16]:
print('C4.5 with prepruning:')
%time C45_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, True)
DECISION_TREE_TESTING('adult.test', C45_prepruning_tree, continuous_indexes, continuous_mid, attributes)

C4.5 with prepruning:
Wall time: 2.88 s
Accuracy:  0.8234752165100424
Precision:  0.9964616003216727
Recall:  0.8140726627685435
F1-score:  0.8960804165461382


In [17]:
print('2-partition with weighting:')
attributes, continuous_indexes = LOAD_ATTRIBUTES("adult.names")
training_examples = LOAD_TRAINING_EXAMPLES("adult.data", True)
%time continuous_mid = CONTINUOUS_VALUE_PROCESSING(continuous_indexes, training_examples, attributes, True)
TRAINING_EXAMPLES_UPDATE(training_examples, continuous_indexes, continuous_mid, attributes)

2-partition with weighting:
Wall time: 43.4 s


In [18]:
print('ID3:')
%time ID3_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, False)
DECISION_TREE_TESTING('adult.test', ID3_tree, continuous_indexes, continuous_mid, attributes)

ID3:
Wall time: 6.01 s
Accuracy:  0.47102296372362024
Precision:  0.5348233782303043
Recall:  0.7571421748930366
F1-score:  0.6268546940713602


In [19]:
print('C4.5:')
%time C45_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, False)
DECISION_TREE_TESTING('adult.test', C45_tree, continuous_indexes, continuous_mid, attributes)

C4.5:
Wall time: 5.05 s
Accuracy:  0.578071783195829
Precision:  0.7960493912683765
Recall:  0.5764327842241698
F1-score:  0.6686702023275607


In [20]:
print('ID3 with prepruning:')
%time ID3_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN, True)
DECISION_TREE_TESTING('adult.test', ID3_prepruning_tree, continuous_indexes, continuous_mid, attributes)

ID3 with prepruning:
Wall time: 365 ms
Accuracy:  0.7637737239727289
Precision:  1.0
Recall:  0.7637737239727289
F1-score:  0.8660676974508984


In [21]:
print('C4.5 with prepruning:')
%time C45_prepruning_tree = DECISION_TREE_LEARNING(training_examples[:], attributes[:], GAIN_RATIO, True)
DECISION_TREE_TESTING('adult.test', C45_prepruning_tree, continuous_indexes, continuous_mid, attributes)

C4.5 with prepruning:
Wall time: 916 ms
Accuracy:  0.8181595842250077
Precision:  0.9928051802702055
Recall:  0.8115402208717245
F1-score:  0.8930677405436502
